# Lecture 35 – Guest Lecture: Elections and Data

## Data 94, Spring 2021

by Lakshya Jain

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as mpatches

In [ ]:
# Let's scrape the data
url_62 = 'https://en.wikipedia.org/wiki/1962_California_gubernatorial_election#Results_by_county'
url_14 = 'https://en.wikipedia.org/wiki/2014_California_gubernatorial_election#Results_by_county'

df_62=pd.read_html(url_62, header=0)[29]
df_14=pd.read_html(url_14, header=0)[35]

In [ ]:
df_14["Brown"] = df_14["Brown"]/(df_14["Brown"] + df_14["Kashkari"]) * 100
df_62["Brown"] = df_62["Brown"].map(lambda x: x.strip("%")).astype("double")
df_62["Nixon"] = df_62["Nixon"].map(lambda x: x.strip("%")).astype("double")
df_62["Brown"] = df_62["Brown"]/(df_62["Brown"] + df_62["Nixon"]) * 100

In [ ]:
df_14.head()

In [ ]:
df_62.head()

In [ ]:
electoral_results = df_14.merge(df_62, on="County")

In [ ]:
electoral_results.head()

In [ ]:
electoral_results = electoral_results[["County", "Brown_x", "Brown_y"]]
electoral_results = electoral_results.rename(columns={'Brown_x': 'Jerry Brown 2014', 'Brown_y': 'Pat Brown 1962'})
electoral_results["Swing"] = electoral_results['Jerry Brown 2014'] - electoral_results['Pat Brown 1962']

In [ ]:
electoral_results.head()

In [ ]:
map_df = gpd.read_file("data/CA_Counties/CA_Counties_TIGER2016.shp")
map_df = map_df.to_crs(epsg=3857)
map_df = map_df.merge(electoral_results[["County", "Swing"]], left_on="NAME", right_on="County", how="inner")

In [ ]:
bins = [-30, -20, -15, -10, -5, 0, 5, 10, 15, 20, 30]
map_df['Bucket'] = 0.0
bin_names = ["Pat Brown > +30"]
for i in range(len(bins)):
    name = ""
    if bins[i] < 0:
        name = "Pat Brown"
        boundary = bins[i+1]
    else:
        name = "Jerry Brown"
        boundary = bins[i]
    bin_names += [name + ' > +' + str(abs(boundary))]
    map_df.loc[map_df['Swing'] > boundary, "Bucket"] = i

map_df['coords'] = map_df['geometry'].apply(lambda x: x.representative_point().coords[:] if x else None)
map_df['coords'] = [coords[0] if coords else None for coords in map_df['coords']]

In [ ]:
map_df.head()

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))
cmap = plt.cm.get_cmap("PiYG", len(bin_names))
ax = map_df.plot(column="Bucket", cmap=cmap,
                   edgecolor="grey", linewidth=0.2, ax=ax)

ax.legend([mpatches.Patch(color=cmap(b)) for b in range(len(bin_names))], bin_names, loc=(1.02, .18))

for idx, row in map_df.iterrows():
    if not row['coords']:
        continue
    plt.annotate(s=row['NAME'], xy=row['coords'], horizontalalignment='center', size=6)

ax.set_axis_off()

plt.gca().set_axis_off()
plt.subplots_adjust(top = 0.95, bottom = 0.05, right = 0.95, left = 0.05, 
            hspace = 0.05, wspace = 0.05)
plt.margins(0,0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())
plt.gca().yaxis.set_major_locator(plt.NullLocator())

plt.title("Pat Brown vs Jerry Brown: 1962-2014")
plt.figtext(0.99, 0.175, 'Data: Wikipedia', horizontalalignment='right')
plt.show()